In [1]:
import io
import os
import sqlite3
import zipfile
from pathlib import Path

import dask.dataframe as dd
import pandas as pd
import regex
from tqdm import tqdm

In [2]:
DATA_DIR = Path('..') / 'data'
INTERIM_DIR = DATA_DIR / '01_interim'

DB = INTERIM_DIR / 'gazetteer_01_idigbio_2020-03-30.db'
CSV = INTERIM_DIR / 'idigbio_2020-03-30b.csv'

CHUNK = 1_000_000

In [3]:
if CSV.exists():
    os.remove(CSV)

In [ ]:
sql = """
    create table if not exists occ_coreid on occurrence (coreid);
    create table if not exists raw_coreid on occurrence_raw (coreid);
"""

with sqlite3.connect(DB) as cxn:
    cxn.executescript(sql)

In [5]:
sql = """
    select occurrence_raw.*,
        isoCountryCode as idigbio_countrycode,
        json_extract(geoPoint, '$.lat') as idigbio_decimallatitude_wgs84,
        json_extract(geoPoint, '$.lon') as idigbio_decimallongitude_wgs84
    from occurrence_raw
    left join occurrence using (coreid);
    """

In [6]:
with open(CSV, 'a') as out:
    with sqlite3.connect(DB) as cxn:

        reader = pd.read_sql(sql, cxn, chunksize=CHUNK)

        header = True

        for df in tqdm(reader):
            df.to_csv(out, header=header, index=False)

            header = False

61it [22:35, 22.23s/it]


In [7]:
!head -1 "$CSV"

coreid,continent,coordinatePrecision,coordinateUncertaintyInMeters,country,countryCode,county,decimalLatitude,decimalLongitude,geodeticDatum,georeferenceSources,higherGeography,island,islandGroup,locality,locationRemarks,maximumDepthInMeters,maximumElevationInMeters,minimumDepthInMeters,minimumElevationInMeters,municipality,stateProvince,verbatimCoordinateSystem,verbatimCoordinates,verbatimDepth,verbatimElevation,verbatimLocality,waterBody,idigbio_countrycode,idigbio_decimallatitude_wgs84,idigbio_decimallongitude_wgs84
